In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


In [2]:
data_train = pd.read_csv("./train.csv")
data_test = pd.read_csv("./test.csv")

In [3]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data_train = data_train.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])
data_test = data_test.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])

In [5]:
data_train = pd.concat([data_train.drop(columns=["Sex"]),pd.get_dummies(data_train["Sex"], drop_first=True)], axis=1)
data_test = pd.concat([data_test.drop(columns=["Sex"]),pd.get_dummies(data_test["Sex"], drop_first=True)], axis=1)

In [6]:
data_train = pd.concat([data_train.drop(columns=["Embarked"]),pd.get_dummies(data_train["Embarked"], drop_first=True)], axis=1)
data_test = pd.concat([data_test.drop(columns=["Embarked"]),pd.get_dummies(data_test["Embarked"], drop_first=True)], axis=1)


In [7]:
data_train = pd.concat([data_train.drop(columns=["Pclass"]),pd.get_dummies(data_train["Pclass"], drop_first=True)], axis=1)
data_test = pd.concat([data_test.drop(columns=["Pclass"]),pd.get_dummies(data_test["Pclass"], drop_first=True)], axis=1)

In [8]:
mean=np.mean(data_train['Age'])
data_train['Age']=data_train['Age'].fillna(mean)
data_test['Age']=data_test['Age'].fillna(mean)

In [9]:
data_train.corr(method="pearson")


,Survived,Age,SibSp,Parch,Fare,male,Q,S,2,3
Survived,1.000000,-0.069809,-0.035322,0.081629,0.257307,-0.543351,0.003650,-0.155660,0.093349,-0.322308
Age,-0.069809,1.000000,-0.232625,-0.179191,0.091566,0.084153,-0.013855,-0.027121,0.006589,-0.281004
SibSp,-0.035322,-0.232625,1.000000,0.414838,0.159651,-0.114631,-0.026354,0.070941,-0.055932,0.092548
Parch,0.081629,-0.179191,0.414838,1.000000,0.216225,-0.245489,-0.081228,0.063036,-0.000734,0.015790
Fare,0.257307,0.091566,0.159651,0.216225,1.000000,-0.182333,-0.117216,-0.166603,-0.118557,-0.413333
male,-0.543351,0.084153,-0.114631,-0.245489,-0.182333,1.000000,-0.074115,0.125722,-0.064746,0.137143
Q,0.003650,-0.013855,-0.026354,-0.081228,-0.117216,-0.074115,1.000000,-0.496624,-0.127301,0.237449
S,-0.155660,-0.027121,0.070941,0.063036,-0.166603,0.125722,-0.496624,1.000000,0.192061,-0.009511
2,0.093349,0.006589,-0.055932,-0.000734,-0.118557,-0.064746,-0.127301,0.192061,1.000000,-0.565210
3,-0.322308,-0.281004,0.092548,0.015790,-0.413333,0.137143,0.237449,-0.009511,-0.565210,1.000000


In [10]:
y_train = data_train["Survived"]

In [11]:
x_train = data_train.drop("Survived", axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3)

In [12]:
classifier1 = DecisionTreeClassifier(max_depth=10, min_samples_leaf=10)
classifier1.fit(x_train, y_train)
print(classification_report(y_train, classifier1.predict(x_train)))
print(confusion_matrix(y_train, classifier1.predict(x_train)))
print(accuracy_score(y_train, classifier1.predict(x_train)))

              precision    recall  f1-score   support

           0       0.85      0.92      0.89       396
           1       0.84      0.72      0.78       227

   micro avg       0.85      0.85      0.85       623
   macro avg       0.85      0.82      0.83       623
weighted avg       0.85      0.85      0.85       623

[[365  31]
 [ 63 164]]
0.8491171749598716


In [13]:
print(classification_report(y_test, classifier1.predict(x_test)))
print(confusion_matrix(y_test, classifier1.predict(x_test)))
print(accuracy_score(y_test, classifier1.predict(x_test)))

              precision    recall  f1-score   support

           0       0.77      0.97      0.86       153
           1       0.93      0.62      0.74       115

   micro avg       0.82      0.82      0.82       268
   macro avg       0.85      0.79      0.80       268
weighted avg       0.84      0.82      0.81       268

[[148   5]
 [ 44  71]]
0.8171641791044776


In [14]:
DecisionTreeClassifier()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [15]:
classifier2 = RandomForestClassifier()
classifier2.fit(x_train, y_train)
print(classification_report(y_train, classifier2.predict(x_train)))
print(confusion_matrix(y_train, classifier2.predict(x_train)))
print(accuracy_score(y_train, classifier2.predict(x_train)))
print(classification_report(y_test, classifier2.predict(x_test)))
print(confusion_matrix(y_test, classifier2.predict(x_test)))
print(accuracy_score(y_test, classifier2.predict(x_test)))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       396
           1       0.99      0.92      0.95       227

   micro avg       0.96      0.96      0.96       623
   macro avg       0.97      0.95      0.96       623
weighted avg       0.97      0.96      0.96       623

[[393   3]
 [ 19 208]]
0.9646869983948636
              precision    recall  f1-score   support

           0       0.80      0.90      0.84       153
           1       0.83      0.70      0.76       115

   micro avg       0.81      0.81      0.81       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268

[[137  16]
 [ 35  80]]
0.8097014925373134


/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [16]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
classifier3 = GridSearchCV(SVC(gamma='scale'), parameters, cv=2)
classifier3.fit(x_train, y_train)
print(classification_report(y_train, classifier3.predict(x_train)))
print(confusion_matrix(y_train, classifier3.predict(x_train)))
print(accuracy_score(y_train, classifier3.predict(x_train)))
print(classification_report(y_test, classifier3.predict(x_test)))
print(confusion_matrix(y_test, classifier3.predict(x_test)))
print(accuracy_score(y_test, classifier3.predict(x_test)))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       396
           1       0.72      0.70      0.71       227

   micro avg       0.79      0.79      0.79       623
   macro avg       0.78      0.77      0.77       623
weighted avg       0.79      0.79      0.79       623

[[335  61]
 [ 69 158]]
0.7913322632423756
              precision    recall  f1-score   support

           0       0.77      0.87      0.82       153
           1       0.79      0.65      0.71       115

   micro avg       0.78      0.78      0.78       268
   macro avg       0.78      0.76      0.77       268
weighted avg       0.78      0.78      0.77       268

[[133  20]
 [ 40  75]]
0.7761194029850746


In [17]:
classifier4 = XGBClassifier()
classifier4.fit(x_train, y_train)
print(classification_report(y_train, classifier4.predict(x_train)))
print(confusion_matrix(y_train, classifier4.predict(x_train)))
print(accuracy_score(y_train, classifier4.predict(x_train)))
print(classification_report(y_test, classifier4.predict(x_test)))
print(confusion_matrix(y_test, classifier4.predict(x_test)))
print(accuracy_score(y_test, classifier4.predict(x_test)))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90       396
           1       0.89      0.74      0.81       227

   micro avg       0.87      0.87      0.87       623
   macro avg       0.88      0.84      0.86       623
weighted avg       0.87      0.87      0.87       623

[[375  21]
 [ 59 168]]
0.8715890850722311
              precision    recall  f1-score   support

           0       0.77      0.95      0.85       153
           1       0.91      0.63      0.74       115

   micro avg       0.81      0.81      0.81       268
   macro avg       0.84      0.79      0.80       268
weighted avg       0.83      0.81      0.81       268

[[146   7]
 [ 43  72]]
0.8134328358208955
